In [1]:
import os
import gc
import sys

## 데이터 관련 라이브러리 로드 

import pandas as pd 
import numpy as np
import re
from tqdm import tqdm

from datasets import Dataset, DatasetDict, load_dataset, concatenate_datasets


In [2]:
## LLM, 딥러닝  관련 라이브러리 로드 

import torch 

from transformers import AutoTokenizer #토크나이저 
from transformers import LlamaForCausalLM,  AutoModelForCausalLM
 # LLM 모델 
from transformers import BitsAndBytesConfig # 양자화 라이브러리 
from transformers import GenerationConfig
from transformers import DataCollatorForLanguageModeling

from peft import PeftModel
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training # 효율적 학습을 위한 라이브러리 , LORA 관련 라이브러리 
from transformers import Trainer, TrainingArguments # 학습 관련된 모델 

In [5]:
#load dataset
import json

with open("../data/dataset_2024_3_28_0.json", "r") as f:
    rawData = json.load(f)

In [8]:
#sample data
print(rawData[0]['text'])

<|assistant|>
학생 반갑습니다.</s>
<|user|>
안녕하세요</s>
<|assistant|>
학생이라고 불러도 되지요?</s>
<|user|>
네</s>
<|assistant|>
방학한 지 1주일 정도 됐나요? 좀 쉬었나요?</s>
<|user|>
네 이제 딱 1주일 됐어요. 여행도 다녀오고 많이 쉬었어요</s>
<|assistant|>
여행은 어디로 다녀왔어요?</s>
<|user|>
가평으로 친구들이랑 다녀왔어요</s>
<|assistant|>
정말 재미있었겠네요</s>
<|assistant|>
같이 간 친구들은 모두 친한 친구들인가요?</s>
<|user|>
네 중학교 때부터 친한 친구들이요</s>
<|assistant|>
학생은 오래되고 친한 친구들이 많군요</s>
<|user|>
네</s>
<|assistant|>
친구들이 학생의 장점에 대해 말해 준 적이 있나요?</s>
<|user|>
친화력이 좋다고 들었던 거 같아요</s>
<|assistant|>
친구들 사이에서 분위기를 좋게 만들어주는 역할을 많이 했나요?</s>
<|user|>
네 보통 많이 하는 편이에요</s>
<|assistant|>
네 [이모티콘] 혹시 MBTI를 해봤어요?</s>
<|user|>
네</s>
<|assistant|>
뭐로 나왔나요?</s>
<|user|>
ESFP 나왔어요</s>
<|assistant|>
네 [이모티콘] 그렇군요. 친한 친구 외에도 많은 친구가 있을 거 같아요. 혹시 단점에 대해서도 들어본 적이 있나요?</s>
<|user|>
잘 모르겠어요</s>
<|assistant|>
[이모티콘] 그런 말은 잘 들어본 적이 없군요. 그러면 학생이 생각할 때 나 자신이 좀 바꾸어 봤으면 하는 부분이 있을까요?</s>
<|user|>
계획을 하긴 하는데 실천을 안 할 때도 많아서 그런 점이 바뀌도록 노력 중이에요</s>
<|assistant|>
실천을 하지 않는 원인이 되는 일은 어떤 것이 있을까요?</s>
<|user|>
여름방학 하루 공부량을 정해놓고

In [ ]:
from transformers import AutoTokenizer

model_id = "beomi/OPEN-SOLAR-KO-10.7B"

tokenizer = AutoTokenizer.from_pretrained(model_id)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048

# Set chat template
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

In [ ]:
## 사용한 Base model 


base_model = 'beomi/OPEN-SOLAR-KO-10.7B'


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
bnb_4bit_compute_dtype = "bfloat16"
use_4bit = True

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
compute_dtype